In [36]:
#!pip install apt-get
#!pip install sqlite3
#!pip install sqlalchemy
#!pip install psycopg2

A **Insurance All** é uma empresa que fornece seguro de saúde para seus clientes e o time de produtos está analisando a possibilidade de oferecer aos assegurados, um novo produto: Um seguro de automóveis.

Assim como o seguro de saúde, os clientes desse novo plano de seguro de automóveis precisam pagar um valor anualmente à Insurance All para obter um valor assegurado pela empresa, destinado aos custos de um eventual acidente ou dano ao veículo.

A Insurance All fez uma pesquisa com cerca de 380 mil clientes sobre o interesse em aderir a um novo produto de seguro de automóveis, no ano passado. Todos os clientes demonstraram interesse ou não em adquirir o seguro de automóvel e essas respostas ficaram salvas em um banco de dados junto com outros atributos dos clientes.

O time de produtos selecionou 127 mil novos clientes que não responderam a pesquisa para participar de uma campanha, no qual receberão a oferta do novo produto de seguro de automóveis. A oferta será feita pelo time de vendas através de ligações telefônicas.

Contudo, o time de vendas tem uma capacidade de realizar 20 mil ligações dentro do período da campanha.

Nesse contexto, você foi contratado como um consultor de Ciência de Dados para construir um modelo que prediz se o cliente estaria ou não interessado no seguro de automóvel. 

Com a sua solução, o time de vendas espera conseguir priorizar as pessoas com maior interesse no novo produto e assim, otimizar a campanha realizando apenas contatos aos clientes mais propensos a realizar a compra.

Como resultado da sua consultoria, você precisará entregar um relatório contendo algumas análises e respostas às seguintes perguntas:

1) Principais Insights sobre os atributos mais relevantes de clientes interessados em adquirir um seguro de automóvel.

2) Qual a porcentagem de clientes interessados em adquirir um seguro de automóvel, o time de vendas conseguirá contatar fazendo 20.000 ligações?

3) E se a capacidade do time de vendas aumentar para 40.000 ligações, qual a porcentagem de clientes interessados em adquirir um seguro de automóvel o time de vendas conseguirá contatar?

4) Quantas ligações o time de vendas precisa fazer para contatar 80% dos clientes interessados em adquirir um seguro de automóvel?

**Business Problem**

    1) Principais Insights sobre os atributos mais relevantes de clientes interessados em adquirir um seguro de automóvel.


**Final Product**
1. Principais insights dos atributos dos clientes

2. Formato da entrega
    Gráficos/Tabela
    
3. Local da entrega
    Telegram/PowerBI


**Process**
1. Passo a passo
    - Realizar feature engineering na base de dados (1º bd - 380K) dos clientes interessados e levantar as hipóteses para cada atributo a ser validada; 
    - No EDA validar as hipóteses e identificar insights dos clientes interessados;


2. Definir o formato da entrega (visualizaçao, tabela, frase)
    - Gráficos que melhor demonstrem as informaçoes.
    - Utilizar os seguintes parâmetros: Id | Gender | Age | Driving License | Region Code | Previously Insured | Vehicle Age | Vehicle Damage | Anual Premium | Policy sales channel | Vintage |
    - Definiçao do schema: colunas e seu tipo
    - Definição da infraestrutura de armazenamento (SQLITE3)
    - Design do ETL (Scripts de Extração, Transformação e Carga)
    - Planejamento de agendamento dos scripts (dependência entre os scripts)
    - Fazer as visualizações
    - Entrega do produto final


3. Decidir o local de entrega (powerBi, telegram, email, streamlit, intranet)
    - PowerBI, Telegram
    
    
**Enter**
1. Fonte de dados:
    - Site da Kaggle
    - BD na AWS

2. Ferramentas:
    - Pyhton 3.8.0
    - AWS
    - Pycharm
    - Jupyter Notebook (análises e prototipagens)
    - Crontjob, Airflow
    - Streamlit
    - PowerBI
    - Telegram

**Business Problem**

    2) Qual a porcentagem de clientes interessados em adquirir um seguro de automóvel, o time de vendas conseguirá contatar fazendo 20.000 ligações?
    3) E se a capacidade do time de vendas aumentar para 40.000 ligações, qual a porcentagem de clientes interessados em adquirir um seguro de automóvel o time de vendas conseguirá contatar?
    4) Quantas ligações o time de vendas precisa fazer para contatar 80% dos clientes interessados em adquirir um seguro de automóvel?


**Final Product**
1. Percentual de clientes interessados da base e se poderá atender o limite de 20K/40K ligaçoes

2. Formato da entrega
    Gráfico ou tabela
    
3. Local da entrega
    Telegram


**Process**
1. Passo a passo
    - Realizar análise da base de dados (2 bd 127 K) e aplicar modelos de ML para obter os atributos que definem
        o cliente com maior probabilidade de adquirir o seguro. Com essas características definir a quantidade 
        de clientes potenciais e se e possível atender com 20/40K ligaçoes.

2. Definir o formato da entrega (visualizaçao, tabela, frase)
    - Gráfico ou tabela

3. Decidir o local de entrega (powerBi, telegram, email, streamlit, intranet)
    - PowerBI, Telegram
    
    
**Enter**
1. Fonte de dados:
    - Site da Kaggle
    - BD na AWS

2. Ferramentas:
    - Pyhton 3.8.0
    - AWS
    - Pycharm
    - Jupyter Notebook (análises e prototipagens)
    - Crontjob, Airflow
    - Streamlit
    - PowerBI
    - Telegram

In [37]:
import csv

import numpy         as np
import pandas        as pd
import psycopg2      as pg 
import pandas.io.sql as psql

In [38]:
#credentials
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port  = 5432
database = 'comunidadedsdb'
user = 'member'
password = 'cdspa'

In [39]:
#connect
conn = pg.connect(host=host,
                 port=port,
                 database=database,
                 user=user,
                 password=password)

# 1.0 Schema Query

In [40]:
cursor = conn.cursor()

query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace

"""

cursor.execute(query_schema)

In [41]:
records = cursor.fetchall()
print(records)
#cursor.close()
#conn.close()

[('pg_toast',), ('pg_temp_1',), ('pg_toast_temp_1',), ('pg_catalog',), ('information_schema',), ('public',), ('pa004',), ('pg_temp_4',), ('pg_toast_temp_4',), ('pa005',), ('pg_temp_9',), ('pg_toast_temp_9',)]


# 2.0 Table Query

In [42]:
cursor = conn.cursor()

query_tables = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname = 'pa004'

"""

cursor.execute(query_tables)

In [43]:
records = cursor.fetchall()
print(records)
#cursor.close()
#conn.close()

[('users',), ('vehicle',), ('insurance',)]


# 3.0 Collect Data using Psycopg2

In [44]:
cursor = conn.cursor()

query_tables_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age >40
    limit 10

"""

cursor.execute(query_tables_users)

In [45]:
records = cursor.fetchall()
print(records)
#cursor.close()
#conn.close()

[(1, 'Male', 44, 28.0, 26.0), (2, 'Male', 76, 3.0, 26.0), (3, 'Male', 47, 28.0, 26.0), (8, 'Female', 56, 28.0, 26.0), (11, 'Female', 47, 35.0, 124.0), (13, 'Female', 41, 15.0, 14.0), (14, 'Male', 76, 28.0, 13.0), (15, 'Male', 71, 28.0, 30.0), (19, 'Male', 42, 28.0, 124.0), (20, 'Female', 60, 33.0, 124.0)]


In [46]:
data = pd.DataFrame(records)
data.head()

0       1   2     3      4
0   1    Male  44  28.0   26.0
1   2    Male  76   3.0   26.0
2   3    Male  47  28.0   26.0
3   8  Female  56  28.0   26.0
4  11  Female  47  35.0  124.0

# 4.0 Collect using Pandas

In [47]:
query_tables_users = """
    SELECT *
    FROM pa004.users u
"""

df_users = psql.read_sql(query_tables_users,conn)
df_users.head()

id  gender  age  region_code  policy_sales_channel
0   1    Male   44         28.0                  26.0
1   2    Male   76          3.0                  26.0
2   3    Male   47         28.0                  26.0
3   4    Male   21         11.0                 152.0
4   5  Female   29         41.0                 152.0

In [48]:
query_tables_vehicle = """
    SELECT *
    FROM pa004.vehicle v
"""

df_vehicle = psql.read_sql(query_tables_vehicle,conn)
df_vehicle.head()

id  driving_license vehicle_age vehicle_damage
0   1                1   > 2 Years            Yes
1   2                1    1-2 Year             No
2   3                1   > 2 Years            Yes
3   4                1    < 1 Year             No
4   5                1    < 1 Year             No

In [49]:
query_tables_insurance = """
    SELECT *
    FROM pa004.insurance i
"""

df_insurance = psql.read_sql(query_tables_insurance,conn)
df_insurance.head()

id  previously_insured  annual_premium  vintage  response
0   1                   0         40454.0      217         1
1   2                   0         33536.0      183         0
2   3                   0         38294.0       27         1
3   4                   1         28619.0      203         0
4   5                   1         27496.0       39         0

In [50]:
query_tables_all = """
    SELECT *
    FROM pa004.insurance i inner join pa004.vehicle v on (i.id = v.id)
                           inner join pa004.users u on (i.id = u.id)
"""

df = psql.read_sql(query_tables_all,conn)


In [51]:
df.to_csv('data.csv')

In [87]:
df = pd.read_csv('data.csv')
df = df.drop(['Unnamed: 0', 'id.1', 'id.2'], axis=1)

# 1.0. Data Description

In [88]:
df1 = df.copy()

## 1.1. Data Dimensions

In [89]:
print('Number of columns: ', df1.shape[1])
print('Number of rows: ', df1.shape[0])

Number of columns:  12
Number of rows:  381109


## 1.2. Data Types

In [90]:
df1.dtypes

id                        int64
previously_insured        int64
annual_premium          float64
vintage                   int64
response                  int64
driving_license           int64
vehicle_age              object
vehicle_damage           object
gender                   object
age                       int64
region_code             float64
policy_sales_channel    float64
dtype: object

## 1.3. Checking NA

In [91]:
df1.isna().sum()

id                      0
previously_insured      0
annual_premium          0
vintage                 0
response                0
driving_license         0
vehicle_age             0
vehicle_damage          0
gender                  0
age                     0
region_code             0
policy_sales_channel    0
dtype: int64

## 1.4. Unity Change

In [92]:
#annual_premium
df1['annual_premium'] = df1['annual_premium'].astype(int)

In [93]:
#region_code
df1['region_code'] = df1['region_code'].astype(int)

In [94]:
#policy_sales_channel
df1['policy_sales_channel'] = df1['policy_sales_channel'].astype(int)

In [95]:
#vehicle_damage
df1['vehicle_damage'] = df1['vehicle_damage'].apply(lambda x: 1 if x == 'Yes' else 0)

In [96]:
#gender
df1['gender'] = df1['gender'].apply(lambda x: 1 if x == 'Female' else 2)

In [97]:
#vehicle_age
df1['vehicle_age'] = df1['vehicle_age'].apply(lambda x: 1 if x == '< 1 Year' else x)
df1['vehicle_age'] = df1['vehicle_age'].apply(lambda x: 1.5 if x == '1-2 Year' else x)
df1['vehicle_age'] = df1['vehicle_age'].apply(lambda x: 2 if x == '> 2 Years' else x)

#transform to (int)
#df1['vehicle_age'] = pd.to_numeric(df1['vehicle_age'],errors = 'coerce')

In [98]:
df1

id  previously_insured  annual_premium  vintage  response  \
0            7                   0           23367      249         0   
1           13                   1           31409      221         0   
2           18                   1           46622      299         0   
3           31                   0            2630      136         0   
4           39                   0           42297      264         0   
...        ...                 ...             ...      ...       ...   
381104  381070                   1           38254      127         0   
381105  381079                   0           39740      170         0   
381106  381082                   1           51619       80         0   
381107  381097                   0           34541       44         0   
381108  381100                   0           44504       71         0   

        driving_license  vehicle_age  vehicle_damage  gender  age  \
0                     1          1.0               1       2   23   
1                     1          1.5               0       1   41   
2                     1          1.0               0       1   25   
3                     1          1.0               0       1   26   
4                     1          1.5               1       2   45   
...                 ...          ...             ...     ...  ...   
381104                1          1.0               0       2   24   
381105                1          1.5               1       1   58   
381106                1          1.5               0       1   75   
381107                1          1.5               1       1   49   
381108                1          1.5               1       1   51   

        region_code  policy_sales_channel  
0                11                   152  
1                15                    14  
2                35                   152  
3                 8                   160  
4                 8                   124  
...             ...                   ...  
381104            8                   152  
381105           28                    26  
381106           28                    26  
381107           10                    26  
381108           28                    26  

[381109 rows x 12 columns]

In [99]:
df1.dtypes

id                        int64
previously_insured        int64
annual_premium            int32
vintage                   int64
response                  int64
driving_license           int64
vehicle_age             float64
vehicle_damage            int64
gender                    int64
age                       int64
region_code               int32
policy_sales_channel      int32
dtype: object

## 1.5. Descriptive Statiscal

In [100]:
#central tendency - mean, median
ct1 = pd.DataFrame(df1.apply(np.mean)).T
ct2 = pd.DataFrame(df1.apply(np.median)).T

#Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(df1.apply(min)).T
d2 = pd.DataFrame(df1.apply(max)).T
d3 = pd.DataFrame(df1.apply(lambda x: x.max() - x.min())).T
d4 = pd.DataFrame(df1.apply(np.std)).T
d5 = pd.DataFrame(df1.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(df1.apply(lambda x: x.kurtosis())).T

#concatenate
m = pd.concat([d1,d2,d3,ct1,ct2,d4,d5,d6]).T.reset_index()
m.columns = ['attributes','min','max','range','mean','median','std','skew','kurtosis']
m

attributes     min       max     range           mean    median  \
0                     id     1.0  381109.0  381108.0  190555.000000  190555.0   
1     previously_insured     0.0       1.0       1.0       0.458210       0.0   
2         annual_premium  2630.0  540165.0  537535.0   30564.389581   31669.0   
3                vintage    10.0     299.0     289.0     154.347397     154.0   
4               response     0.0       1.0       1.0       0.122563       0.0   
5        driving_license     0.0       1.0       1.0       0.997869       1.0   
6            vehicle_age     1.0       2.0       1.0       1.304808       1.5   
7         vehicle_damage     0.0       1.0       1.0       0.504877       1.0   
8                 gender     1.0       2.0       1.0       1.540761       2.0   
9                    age    20.0      85.0      65.0      38.822584      36.0   
10           region_code     0.0      52.0      52.0      26.388807      28.0   
11  policy_sales_channel     1.0     163.0     162.0     112.034295     133.0   

              std          skew    kurtosis  
0   110016.691870 -8.071174e-18   -1.200000  
1        0.498251  1.677471e-01   -1.971871  
2    17213.132474  1.766087e+00   34.004569  
3       83.671194  3.029517e-03   -1.200688  
4        0.327935  2.301906e+00    3.298788  
5        0.046109 -2.159518e+01  464.354302  
6        0.283719  2.528959e-01   -0.792253  
7        0.499976 -1.950724e-02   -1.999630  
8        0.498336 -1.635904e-01   -1.973249  
9       15.511591  6.725390e-01   -0.565655  
10      13.229871 -1.152664e-01   -0.867857  
11      54.203924 -9.000081e-01   -0.970810

# 2.0. Feature Engineering

In [101]:
df2 = df1.copy()